# Base Model

This will be the base model for the Spotify data sets. The goal is to create a playlist of 30 songs that maximizes user ratings, does not include the same song twice, and does not repeat artists!

### Decision Variables

##### Binary Variables:

- Let $X_{i}$ be the binary variable such that it is 1 if the song exists in the playlist, and 0 otherwise!
- Let $Y_{i}$ be the binary variable such that it is 1 if the the artist is in the playlist, and 0 otherwise!

##### Additional Data:

- Let there be a list of ratings "ratings"
- Let there be a list of possible track names "track_names" (i)
- Let there be a list of possible artist names "artist_to_song" (j)
- Let there be a list of the average ratings "rating"
- Let unique_artists be the set of unique artist names

### Constraints

- There can only be 30 songs
 $$ \sum_{i=1}^{n} X_i \;=\; 30 $$

- No artist can be repeated twice
  $$\forall\,j \in \{0,1,\ldots,|\text{unique\_artists}|-1\}:\quad \sum_{i \in \text{artist\_to\_songs}[j]} X_i \leq Y_j \quad \text{(link constraint)}$$
  $$\forall\,j \in \{0,1,\ldots,|\text{unique\_artists}|-1\}:\quad \sum_{i \in \text{artist\_to\_songs}[j]} X_i \leq 1 \quad \text{(uniqueness constraint)}$$

- No song can be repeated twice
   $$ \forall\,i=1,\dots,n:\quad X_i \;\le\; 1 $$

### Objective Function

Maximize the user ratings to get the best songs!
$$\max \;\sum_{i=1}^{n} \bigl(\text{rating}_i \times X_i\bigr) $$

In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np 

In [2]:
# load the artists data set
artists = pd.read_csv('artists.csv')

# load the songs data set
#songs = pd.read_csv('songs_with_predictions_small.csv')
songs = pd.read_csv('Final_Cleaned_Song_Predictions.csv')

/var/folders/jf/ld91zbl12kd2wx8cvq_4pqz00000gn/T/ipykernel_81870/1694117371.py:2: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  artists = pd.read_csv('artists.csv')


In [3]:
def user_choice(songs_dict):
    valid_user_ids = [col for col in songs_dict.keys() if col.startswith("user_")]
    print("Please enter the user ID for which you want to see the ratings:")
    while True:
        user_id = input()
        if user_id in valid_user_ids:
            return user_id
        else:
            print("Invalid user ID. Please enter a valid user ID from the dataset.")

In [ ]:
songs_dict = songs.to_dict(orient="list")

# call the function user_choice to get the target_user that the person wants to see
target_user = user_choice(songs_dict)
ratings = songs_dict[target_user]

track_names = songs_dict['track_name']
num_tracks = len(track_names)

artists = songs_dict['artist_name']
unique_artists = list(set(artists))

Please enter the user ID for which you want to see the ratings:
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid user ID from the dataset.
Invalid user ID. Please enter a valid us

In [135]:
# create a model to recommend songs for this playlist
model = Model("Playlist Recommendation")

# create a binary variable, X_i, that is 1 if song i is selected, 0 otherwise
X = model.addVars(num_tracks, vtype=GRB.BINARY, name="X")

# create a binary variable, Y_i, that is 1 if artist i is selected, 0 otherwise
Y = model.addVars(len(unique_artists), vtype=GRB.BINARY, name="Y")

# the first constraint is that there must be 30 songs
model.addConstr(quicksum(X[i] for i in range(num_tracks)) == 30, "num_songs")

# the second constraint is that no artist can be repeated twice
artist_to_songs = {artist: [] for artist in unique_artists}
for i, artist in enumerate(artists):
    artist_to_songs[artist].append(i)

for j, artist in enumerate(unique_artists):
    model.addConstr(quicksum(X[i] for i in artist_to_songs[artist]) <= Y[j], f"artist_{artist}_link")
    model.addConstr(quicksum(X[i] for i in artist_to_songs[artist]) <= 1, f"artist_{artist}_unique")

# the third constraint is that no songs can be repeated
model.addConstrs((X[i] <= 1 for i in range(num_tracks)), "no_repeats")


# the objective function is to pick the songs with the highest ratings
model.setObjective(quicksum(X[i] * ratings[i] for i in range(num_tracks)), GRB.MAXIMIZE)


In [136]:
from tabulate import tabulate

# optimize the model
model.optimize()

# prepare the results in a tabular format
results = []
for idx, i in enumerate(range(num_tracks)):
    if X[i].x > 0:
        results.append([len(results) + 1, track_names[i], artists[i], round(ratings[i], 2)])

# sort the results alphabetically by artist
results.sort(key=lambda x: x[2])

# re-number the songs after sorting
for idx, row in enumerate(results):
    row[0] = idx + 1

# print the results in a pretty table
print("\nRecommended Songs:")
print(tabulate(results, headers=["#", "Song", "Artist", "Rating"], tablefmt="fancy_grid"))


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.4.0 24E248)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34961 rows, 27480 columns and 87480 nonzeros
Model fingerprint: 0x6887c31a
Variable types: 0 continuous, 27480 integer (27480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -26.8592291
Presolve removed 34961 rows and 27480 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.03 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 0.127979 -26.8592 
No other solutions better than 0.127979

Optimal solution found (tolerance 1.00e-04)
Best objective 1.279786049101e-01, best bound 1.279786049101e-01, gap 0.0000%

Recommended Songs:
╒═══

Now check 10 best solutions && compare, help from hw 3